In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

# from hloc import extract_features, match_features, reconstruction, visualization
# from hloc import colmap_for_habitat, triangulation, localize_sfm, visualization
import h5py
import os
import numpy as np
from collections import defaultdict
import pycolmap
import matplotlib
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation as R
import math
import g2o
from enum import Enum
import cv2
# from threading import Lock
# from hloc.utils.parsers import (
#     parse_image_lists_with_intrinsics, parse_retrieval, names_to_pair)
from tqdm import tqdm
import json
from time import perf_counter
import time

In [ ]:
dataset = Path('/datasets/Habitat/')
images_path = dataset / 'extracted_HPointLoc'

# outputs = Path('/datasets/Habitat/Hierarchical_Localization_outputs/sfm/') 
# loc_pairs = dataset / 'FBoW/scores_FBoW_all_maps.txt'  # top 50 retrieved by NetVLAD

# feature_filename = f"{features}.h5"
# match_filename = f"{features}_{matcher_conf['output']}_{loc_pairs.stem}.h5"

path_to_hdf5_datasets = '/datasets/Habitat/HPointLoc/1LXtFkjw3qL_point0'

# Getting table of localizations

In [ ]:
def quaternion_to_rotation_matrix(quaternion_wxyz):
    r = R.from_quat([quaternion_wxyz[1], quaternion_wxyz[2], quaternion_wxyz[3], quaternion_wxyz[0]])
    matrix = r.as_matrix()
    matrix[:3,2] = -matrix[:3,2]
    matrix[:3,1] = -matrix[:3,1]
    return matrix

In [ ]:
def get_point_3d(x, y, depth, fx, fy, cx, cy, cam_center_world, R_world_to_cam, w_in_quat_first = True):
    if depth <= 0:
        return 0
    new_x = (x - cx)*depth/fx
    new_y = (y - cy)*depth/fy
    new_z = depth
    coord_3D_world_to_cam = np.array([new_x, new_y, new_z], float)
    if len(R_world_to_cam) == 4:
        if w_in_quat_first:
            matrix = quaternion_to_rotation_matrix(R_world_to_cam)
        else:
            R_world_to_cam = [R_world_to_cam[3], R_world_to_cam[0], R_world_to_cam[1], R_world_to_cam[2]]
            matrix = quaternion_to_rotation_matrix(R_world_to_cam)
    coord_3D_cam_to_world = np.matmul(matrix, coord_3D_world_to_cam) + cam_center_world
    return coord_3D_cam_to_world

In [ ]:
distances = np.array([0, 0.25, 0.5, 1, 2, 3, 4, 5, 10, 20, 30, 50])
angles = np.array([0, 1, 2, 5, 10, 15, 20, 30, 40, 50, 60, 70, 80, 90, 120, 150, 180], dtype=float)

table = {}
for distance in distances:
    if distance == 0:
        continue
    for angle in angles:
        if angle == 0:
            continue
        table[str(distance)+" m and "+ str(angle) + " deg"] = {
            'tp'           : 0,
            'all'           : 0,
            'queries-dbs' : {}
        }

## Creating empty table

In [ ]:
# optimizer = BundleAdjustment()

for filename1 in tqdm(sorted(os.listdir(path_to_images))):
    query1_filename = filename1.rstrip('.png')
    hdf5_filename_query1 = '_'.join(query1_filename.split('_')[:2]) + '.hdf5'
    hdf5_file_query1 = h5py.File(os.path.join(path_to_hdf5_datasets, hdf5_filename_query1), 'r')
    num_query1 = int(query1_filename.split('_')[-1])
    
    is_database = ''
    if query1_filename.find('database') != -1:
        is_database = '_base'
    
    depth_query1 = 10*hdf5_file_query1['depth'+is_database][num_query1]
    
    translation = hdf5_file_query1['gps'+is_database][num_query1]
    orientation = quaternion_to_rotation_matrix(hdf5_file_query1['quat'+is_database][num_query1])
    pose_44_query1 = np.eye(4)
    pose_44_query1[:3,:3] = orientation
    pose_44_query1[:3,3] = translation
    
    # Заполняем таблицу метрик    
    for filename2 in sorted(os.listdir(path_to_images)):
        if filename1 == filename2:
            continue
        query2_filename = filename2.rstrip('.png')
        hdf5_filename_query2 = '_'.join(query2_filename.split('_')[:2]) + '.hdf5'
        hdf5_file_query2 = h5py.File(os.path.join(path_to_hdf5_datasets, hdf5_filename_query2), 'r')
        
        num_query2 = int(query2_filename.split('_')[-1])
        
        is_database = ''
        if query2_filename.find('database') != -1:
            is_database = '_base'
        
        translation = hdf5_file_query2['gps'+is_database][num_query2]
        orientation = quaternion_to_rotation_matrix(hdf5_file_query2['quat'+is_database][num_query2])
        pose_44_query2 = np.eye(4)
        pose_44_query2[:3,:3] = orientation
        pose_44_query2[:3,3] = translation
        
        diff_pose = np.linalg.inv(pose_44_query2) @ pose_44_query1
        dist_diff = np.sum(diff_pose[:3, 3]**2) ** 0.5
        r = R.from_matrix(diff_pose[:3, :3])
        rotvec = r.as_rotvec()
        angle_diff = (np.sum(rotvec**2)**0.5) * 180 / 3.14159265353
        angle_diff = abs(180 - abs(angle_diff-180))
        
        num_dist =  [i for i in range(1, len(distances)-1) \
                     if (distances[i-1] < dist_diff) and (distances[i] >= dist_diff)]
        
        num_angle = [i for i in range(1, len(angles)-1) \
                     if (angles[i-1] < angle_diff) and (angles[i] >= angle_diff)]
        
        # Если не входит в таблицу, то пропускаем эту пару
        if len(num_dist) == 0 or len(num_angle) == 0:
            continue
        
        if query1_filename in table[str(distances[num_dist[0]])+" m and " + str(angles[num_angle[0]])+" deg"]['queries-dbs'].keys():
            table[str(distances[num_dist[0]])+" m and " + str(angles[num_angle[0]])+" deg"]['queries-dbs'][query1_filename]["db_filenames"].append(query2_filename)
#             table[str(distances[num_dist[0]])+" m and " + str(angles[num_angle[0]])+" deg"]["all"] += 1
        else:
            table[str(distances[num_dist[0]])+" m and " + str(angles[num_angle[0]])+" deg"]['queries-dbs'][query1_filename] = {
                "db_filenames"   : [query2_filename]
            }
#             table[str(distances[num_dist[0]])+" m and " + str(angles[num_angle[0]])+" deg"]["all"] += 1

## Filling table with results

In [ ]:
num_keypoints = 4096
path_to_global_descriptors = '/datasets/Habitat/HF_Net_output/global_descriptors'

# conf = confs['superglue']
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
# Model = dynamic_load(matchers, conf['model']['name'])
# model = Model(conf['model']).eval().to(device)

count = 0

for cell in tqdm(table.keys()):
    if len(table[cell]['queries-dbs'].keys()) == 0:
        continue
        
#     if cell != "2.0 m and 10.0 deg":
#         continue
        
    for query_image in table[cell]['queries-dbs'].keys():
        
        # RUN global descriptor retrieval and find top-1 retrieval image
    
        dbFeat = []

        with open(os.path.join(path_to_global_descriptors, query_image+'.npy'), 'rb') as f:
            descriptor_query = np.load(f)
        qFeat = np.empty((1, num_keypoints))
        qFeat[0,:] = descriptor_query
        qFeat = qFeat.astype('float32')
        faiss_index = faiss.IndexFlatL2(num_keypoints)
        faiss_index.add(qFeat)
    
#         measurements = []
#         optimizer.clear()
        for db_image in table[cell]['queries-dbs'][query_image]["db_filenames"]:
        
            with open(os.path.join(path_to_global_descriptors, db_image+'.npy'), 'rb') as f:
                descriptor2 = np.load(f)
            dbFeat.append(descriptor2)
        
        dbFeat_temp = np.empty((len(dbFeat), num_keypoints))

        for i in range(len(dbFeat)):
            dbFeat_temp[i,:] = dbFeat[i]
    
        dbFeat = dbFeat_temp
    
        dbFeat = dbFeat.astype('float32')
        distances, predictions = faiss_index.search(dbFeat, 1)
        num_db = predictions[0][0]
        db_image = table[cell]['queries-dbs'][query_image]["db_filenames"][num_db]
        image_1 = query_image
        image_2 = db_image
        
        dbfeature = dbFeat[num_db]
        qfeature  = qFeat[0]
        dbfeature = np.reshape(dbfeature, (1,-1))
        qfeature = np.reshape(qfeature, (1,-1))
        simil_score = cosine_similarity(dbfeature, qfeature)[0][0]
    
#----------- The following commented is for localization results (SuperPoint + SuperGLue + g2o) ---------------

#         # Run SuperPoint + SuperGlue + G2O
    
#         query1_filename = image_1
#         is_database = ''
#         if query1_filename.find('database') != -1:
#             is_database = '_base'
#         hdf5_filename_query1 = '_'.join(query1_filename.split('_')[:2]) + '.hdf5'
#         hdf5_file_query1 = h5py.File(os.path.join(path_to_hdf5_datasets, hdf5_filename_query1), 'r')
#         num_query1 = int(query1_filename.split('_')[-1])
#         depth_query1 = 10*hdf5_file_query1['depth'+is_database][num_query1]
#         translation = hdf5_file_query1['gps'+is_database][num_query1]
#         orientation = quaternion_to_rotation_matrix(hdf5_file_query1['quat'+is_database][num_query1])
#         pose_44_query1 = np.eye(4)
#         pose_44_query1[:3,:3] = orientation
#         pose_44_query1[:3,3] = translation
#         keypoints_query = keypoints_file[query1_filename]['keypoints'].__array__()
    
#         db_filename = image_2
#         hdf5_filename_db = '_'.join(db_filename.split('_')[:2]) + '.hdf5'
#         hdf5_file_db = h5py.File(os.path.join(path_to_hdf5_datasets, hdf5_filename_db), 'r')
#         num_db = int(db_filename.split('_')[-1])
#         is_database = ''
#         if db_filename.find('database') != -1:
#             is_database = '_base'
#         translation = hdf5_file_db['gps'+is_database][num_db]
#         orientation = quaternion_to_rotation_matrix(hdf5_file_db['quat'+is_database][num_db])
#         pose_44_db = np.eye(4)
#         pose_44_db[:3,:3] = orientation
#         pose_44_db[:3,3] = translation
#         depth_db = 10*hdf5_file_db['depth'+is_database][num_db]
#         keypoints_db = keypoints_file[db_filename]['keypoints'].__array__()
    
#         pose = g2o.Isometry3d(pose_44_db)
#         optimizer.add_pose(0, pose, cam, fixed=False)
    
#         retrieved_image = image_2
#         hdf5_file_retrieved = hdf5_file_db
#         num_retrieved = num_db
#         depth_retrieved = depth_db
        
#         # Extract matches
        
#         data = {}
#         feats0, feats1 = keypoints_file[query_image], keypoints_file[db_image]
#         for k in feats1.keys():
#             data[k+'0'] = feats0[k].__array__()
#         for k in feats1.keys():
#             data[k+'1'] = feats1[k].__array__()
#         data = {k: torch.from_numpy(v)[None].float().to(device)
#                 for k, v in data.items()}
#         data['image0'] = torch.empty((1, 1,)+tuple(feats0['image_size'])[::-1])
#         data['image1'] = torch.empty((1, 1,)+tuple(feats1['image_size'])[::-1])
        
#         pred = model(data)
        
#         matches_db_keypoints_ids = pred['matches0'][0].cpu().short().numpy()
#         matched_keypoints_query_ids = np.where(matches_db_keypoints_ids > -1)[0]
        
#         # Пробегаемся по всем индексам кипоинтов query image, которые имеют сопоставление с database image
#         for num_kp, keypoint_id in enumerate(matched_keypoints_query_ids):
#             if keypoint_id == -1:
#                 continue
#             # затем получаем индекс сопоставленного кипоинта на database image
#             keypoint_id_query = keypoint_id
#             keypoint_id_database = matches_db_keypoints_ids[keypoint_id]
                
#             # получаем координаты x,y для сопоставленных кипоинтов на query и database
#             keypoint_xy_query = keypoints_query[keypoint_id_query]
#             keypoint_xy_db = keypoints_db[keypoint_id_database]
                
#             # получаем глубины данных кипоинтов
#             depth_keypoint_query = depth_query1[int(keypoint_xy_query[1]), int(keypoint_xy_query[0])][0]
#             depth_keypoint_db = depth_retrieved[int(keypoint_xy_db[1]), int(keypoint_xy_db[0])][0]
                
#             # если глубина кипоинтов <=0, то она скорее всего невалидна, пропускаем ее
#             if depth_keypoint_db <= 0 or depth_keypoint_query <= 0:
#                 continue
                    
#             # Получаем координаты виртуального кипоинта для виртуальной правой камеры на query image
#             x_right = keypoint_xy_query[0] - fx_baseline/depth_keypoint_query
                
#             # Создаем объекты кипоинтов (на query image, left and right cam), подходящие для погрузки их в g2o
#             kp_left, kp_right = cv2.KeyPoint(), cv2.KeyPoint()
#             kp_left.pt  = (keypoint_xy_query[0], keypoint_xy_query[1])
#             kp_right.pt = (x_right, keypoint_xy_query[1])
#             meas = Measurement(Measurement.Type.STEREO,
#                                Measurement.Source.TRIANGULATION,
#                                [kp_left, kp_right])
                
#             #  Теперь нужно загрузить в g2o 3D координаты кипоинта на database image. Этот кипоинт сопоставлен
#             # с кипоинтом на query image
#             # Получаем 3D координату ключевой точки на database image
#             global_xyz_of_keypoint = get_point_3d(keypoint_xy_db[0], keypoint_xy_db[1], depth_keypoint_db, \
#                                      fx, fy, cx, cy, \
#                                      translation, hdf5_file_retrieved['quat'+is_database][num_retrieved])
#             mappoint = MapPoint(global_xyz_of_keypoint)
#             # загружаем 3D координату кипоинта в объект meas, который будет теперь содержать координаты (x,y)
#             # киопинта на query image и 3D координаты (x,y,z) кипоинта на database image 
#             meas.mappoint = mappoint
#             measurements.append(meas)
                
#         for i, m in enumerate(measurements):
#             optimizer.add_point(i, m.mappoint.position, fixed=True)
#             optimizer.add_edge(0, i, 0, m)
#         optimizer.optimize(10)
#         result = optimizer.get_pose(0)
    
#         pose_estimated = np.eye(4)
#         # Now get pose from g2o optimization 
#         matrix = result.matrix()[:3,:3]
#         pose_estimated[:3, :3] = matrix
#         pose_estimated[:3, 3] = result.position()
    
#         error_pose = np.linalg.inv(pose_estimated) @ pose_44_query1
#         dist_error = np.sum(error_pose[:3, 3]**2) ** 0.5
#         r = R.from_matrix(error_pose[:3, :3])
#         rotvec = r.as_rotvec()
#         angle_error = (np.sum(rotvec**2)**0.5) * 180 / 3.14159265353
#         angle_error = abs(180 - abs(angle_error - 180))
    
        table[cell]['all'] += 1
#         if angle_error < 5 and dist_error < 0.5:
#             table[cell]['tp'] += 1
            
#         table[cell]['queries-dbs'][query_image]["estimated_pose"] = pose_estimated.tolist()
        table[cell]['queries-dbs'][query_image]["retrieved_image"] = retrieved_image
        table[cell]['queries-dbs'][query_image]["retrival_score"] = simil_score
        table[cell]['queries-dbs'][query_image]["num_matches"] = float(simil_score)
            
# ----------------- The following is for saving images and further making video -----------------------

#         if dist_error > 0.5 or angle_error > 5 and save_images:
            
#             new_image = Image.new('RGB',(2*256, 256), (250,250,250))

#             db_image1 = db_image
        
#             image1 = Image.open(str(images_path / Path(query_image))+'.png')
#             image2 = Image.open(str(images_path / Path(db_image1))+'.png')
        
# #             metric1 = list(retrievals[query_image+'.png'][0].values())[0]
        
#             new_image.paste(image1,(0,0))
#             new_image.paste(image2,(255,0))

#             draw = ImageDraw.Draw(new_image)
        
#             draw.text((0, 0), query_image,(255,255,255), font=font)
#             draw.text((image1.size[0], 0),"{}\n{}".format(db_image.rstrip('.png'), round(float(simil_score),3)),(255,255,255),font=font)

#             new_image.save('{}/{:04}.jpg'.format(output_folder_for_images, count),'JPEG')
#             count += 1
            
#         print(table[cell])
#         print(dist_error)
#         print(angle_error)
#     break

In [ ]:
# Saving the filled table into json

import json
with open('table.json', 'w') as outfile:
    json.dump(table, outfile)

# Printing table

In [2]:
from tabulate import tabulate #pip install tabulate
import json
import numpy as np
import h5py
import os
from scipy.spatial.transform import Rotation as R
from tqdm import tqdm

ModuleNotFoundError: No module named 'tabulate'

In [ ]:
json_data = '/home/cds-s/workspace/hierarchical_localization/table.json'
path_to_hdf5_datasets = '/media/cds-s/data/Datasets/Habitat/1LXtFkjw3qL_point0/hdf5/'
output_path = '/media/cds-s/data/Datasets/Habitat/table.txt'

In [ ]:
with open(json_data) as f:
    table = json.load(f)

In [ ]:
def quaternion_to_rotation_matrix(qvec):
    r = R.from_quat([qvec[1], qvec[2], qvec[3], qvec[0]])
    result = r.as_matrix()
    result[:3,2] = -result[:3,2]
    result[:3,1] = -result[:3,1]
    return result

In [ ]:
angle_thr = 10
dist_thr = 2
table_metrics = {}
for key in tqdm(table.keys()):
    if not(key in table_metrics.keys()):
        table_metrics[key] = {
            'tp'  : 0,
            'all' : 0
        }
    for query in table[key]['queries-dbs']:
        hdf5_filename = '_'.join(query.split('_')[:2]) + '.hdf5'
        num_image = int(query.split('_')[-1])
        hdf5_file = h5py.File(os.path.join(path_to_hdf5_datasets, hdf5_filename), 'r')
        pose_query_gt = np.eye(4)
        is_database = ""
        if query.find('database') != -1:
            is_database = "_base"
        pose_query_gt[:3,:3] = quaternion_to_rotation_matrix(hdf5_file['quat'+is_database][num_image])
        pose_query_gt[:3,3] = hdf5_file['gps'+is_database][num_image]
        pose_query_estimated = table[key]['queries-dbs'][query]['estimated_pose']
        error_pose = np.linalg.inv(pose_query_estimated) @ pose_query_gt
        dist_error = np.sum(error_pose[:3, 3]**2) ** 0.5
        r = R.from_matrix(error_pose[:3, :3])
        rotvec = r.as_rotvec()
        angle_error = (np.sum(rotvec**2)**0.5) * 180 / 3.14159265353
        angle_error = abs(180 - abs(angle_error-180))
        if angle_error < angle_thr and dist_error < dist_thr:
            table_metrics[key]['tp'] += 1
        table_metrics[key]['all'] += 1

In [ ]:
table = table_metrics

### Printing into this jupyter notebook the whole table and into text file

In [ ]:
distances = []
angles = []
for key in table.keys():
    distance, _, _, angle, _ = key.split(' ')
    distances.append(float(distance))
    angles.append(float(angle))
distances = sorted(distances)
angles = sorted(list(set(angles)))[::-1]

distances = sorted(list(set(distances)))

table_for_printing = []

set_of_existing_angles = set()

num_angle = 0
num_distance = 0
stroka = []

output_file = open(output_path, 'w')
while num_angle != len(angles):
    for key in table.keys():
        distance, _, _, angle, _ = key.split(' ')
        distance = float(distance)
        angle = float(angle)
        if angle == angles[num_angle] and distance == distances[num_distance]:
            if num_distance == 0:
                if num_angle != len(angles)-1:
                    stroka = [str(int(angles[num_angle+1])) + "°-\n"+ str(int(angles[num_angle]))+"°"]
                else:
                    stroka = ["0°-\n"+ str(int(angles[num_angle])) + " °"]
                stroka.append(str(table[key]["tp"])+"/"+str(table[key]["all"]))
                num_distance += 1
            else:
                num_distance += 1
                stroka.append(str(table[key]["tp"])+"/"+str(table[key]["all"]))
            if num_distance == len(distances):
                stroka_for_txt_file = stroka[1:]
                stroka_for_txt_file = ' '.join(stroka_for_txt_file)
                output_file.write(stroka_for_txt_file + "\n")
                table_for_printing.append(stroka)
                num_distance = 0
                num_angle += 1
                if num_angle == len(angles):
                    break

# print(stroka)

table_for_printing.append([" "] + ['0-\n'+str(distances[0])+" m"] + [str(distances[i])+"-\n"+str(distance)+" m" for i, distance in enumerate(distances[1:])])

print(tabulate(table_for_printing, tablefmt='fancy_grid'))
output_file.close()

### The same but without printing into jupyter notebook, only into text file

In [ ]:
distances = []
angles = []
for key in table.keys():
    distance, _, _, angle, _ = key.split(' ')
    distances.append(float(distance))
    angles.append(float(angle))
distances = sorted(distances)
angles = sorted(list(set(angles)))[::-1]

distances = sorted(list(set(distances)))

table_for_printing = []

set_of_existing_angles = set()

num_angle = 0
num_distance = 0
stroka = []

output_file = open(output_path, 'w')
while num_angle != len(angles):
    for key in table.keys():
        distance, _, _, angle, _ = key.split(' ')
        distance = float(distance)
        angle = float(angle)
        if angle == angles[num_angle] and distance == distances[num_distance]:
            num_distance += 1
            if table[key]["all"] != 0:
                stroka.append(str(table[key]["tp"]/table[key]["all"]))
            else:
                stroka.append(str(0))
            if num_distance == len(distances):
                stroka_for_txt_file = ' '.join(stroka)
                stroka = []
                output_file.write(stroka_for_txt_file + "\n")
                num_distance = 0
                num_angle += 1
                if num_angle == len(angles):
                    break

output_file.close()